In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing as pp
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')


In [ ]:
all_price = pd.concat([one, two, three, four, top_rest], axis =0)
all_price.info()

In [ ]:
all_price[all_price['id'] == '108']

In [ ]:
all_price['id'] = pd.to_numeric(all_price.id)

In [ ]:
all_price[all_price.price_level.isnull()]

In [ ]:
all_price['price_level'] = all_price['price_level'].fillna(all_price.groupby('city')['price_level'].transform('mean'))

In [ ]:
#Beirut and Tehran are still NA. fill na with 2?
all_price.fillna(2.0, inplace=True)

In [ ]:
all_price.info()

In [ ]:
all_price['price_level'] = all_price['price_level'].astype(int)


In [ ]:
all_price.isnull().all()

In [ ]:
all_price.price_level.fillna(0, inplace=True)

In [ ]:
all_price.price_level.isnull().any()

In [ ]:
all_price.isnull().any()

In [ ]:
all_price

In [ ]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level

In [ ]:
food = to_city(all_price)
food

In [ ]:
food.reset_index(inplace=True)
food.drop(columns = ['avg_price'], inplace=True)
food

In [ ]:
#Drop the second Cancun with no '
#Rename K
food[food['country'] =='Mexico']

In [ ]:
food

In [ ]:
#Dropping the second Cancún
food = food.drop(food[food.id == 40].index)

nan_row = {'country' : None, 'city': 'Zx', 'id': 200, 1.0: 0, 2.0: 0, 3.0: 0, 4.0: 0}
food = food.append(nan_row, ignore_index=True)

In [ ]:
cities = get_df('cities')
cities = cities.drop(cities[cities.city == 'Guilin'].index)


In [ ]:
cities

In [ ]:
new_row = {'id': 200, 'city': 'Zx', 'country': 'None'}

cities = cities.append(new_row, ignore_index=True)

In [ ]:
food

In [ ]:
le = pp.LabelEncoder()
le.fit(cities.city)
#numpy.save('classes.npy', encoder.classes_)

food['label_id'] = le.transform(food.city)
food

In [ ]:
food = food.drop(food[food.id == 200].index)


In [ ]:
#Reverse encoder
#list(le.inverse_transform(food.label_id))

In [ ]:
food_new = food[['label_id', 1.0, 2.0, 3.0, 4.0]].copy()
food_new

In [ ]:
food_new.iloc[23]

In [ ]:
food_new.sort_values('label_id', inplace=True)
food_new.set_index('label_id', inplace=True)

In [ ]:
food_new.fillna(0, inplace=True)

In [ ]:
food_new

In [ ]:
normalized = pp.normalize(food_new)
normalized_city = pd.DataFrame(normalized)
normalized_city

In [ ]:
sim_city_food = pd.DataFrame(cosine_similarity(normalized_city))

In [ ]:
sim_city_food

### Survey / user food data

In [ ]:
survey = get_df('survey_response')

In [ ]:
survey.columns

In [ ]:
survey_food = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_food_only.csv', index_col=0)
survey_food

In [ ]:
replace_map = {'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often': 3, 'Always': 4}

df_food_replace = survey_food.replace(replace_map)

In [ ]:

df_food_replace

In [ ]:
df_food_replace.drop(columns=['nationality', 'age', 'gender'], inplace=True)

In [ ]:
df_food_replace

In [ ]:
normalized_user = pd.DataFrame(pp.normalize(df_food_replace))
normalized_user

In [ ]:
normalized_city

In [ ]:
cosine_sim = pd.DataFrame(cosine_similarity(normalized_user, normalized_city))
cosine_sim

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [ ]:
top_10_city = find_similar_n(cosine_sim,10)
top_10_city

In [ ]:
user_1 = top_10_city.iloc[0, 0:5].values.tolist()
user_1

In [ ]:
for city in user_1:
    city2 = food[food.label_id == city]
    print(city2.city)

# Attraction Similarity

In [ ]:
city_attraction = pd.read_csv('/Users/tristannisbet/Documents/SM/city_attraction_only.csv', index_col=0)
city_attraction

In [ ]:
city_attraction[city_attraction['id'] == 40]

In [ ]:
# Drop Guilin because there were no restaurants found
city_attraction.reset_index(inplace=True)
city_attraction = city_attraction.drop(city_attraction[city_attraction.id == 80].index)
city_attraction = city_attraction.drop(city_attraction[city_attraction.id == 40].index)


In [ ]:
city_attraction.set_index('country', inplace=True)
city_attraction.sort_values('country')

In [ ]:
city_attraction[city_attraction['id'] == 40]

In [ ]:
city_attraction.shape

In [ ]:
city_attraction.sort_values('id', inplace=True)

In [ ]:
for i in city_attraction.id:
    print(i)

In [ ]:
city_attraction['city'] = city_attraction['city'].str.replace('Kraków', 'Krakow')
city_attraction['city'] = city_attraction['city'].str.replace('Muğla', 'Mugla')
city_attraction['city'] = city_attraction['city'].str.replace('São Paulo', 'Sao Paulo')
city_attraction['city'] = city_attraction['city'].str.replace('Zürich', 'Zurich')
city_attraction['city'] = city_attraction['city'].str.replace('Düsseldorf', 'Dusseldorf')
city_attraction['city'] = city_attraction['city'].str.replace('Frankfurt am Main', 'Frankfurt')

In [ ]:
print(le)

In [ ]:
#le = pp.LabelEncoder()

city_attraction['label_id'] = le.transform(city_attraction.city)
city_attraction

In [ ]:
city_attraction.sort_values('label_id', inplace=True)
city_attraction_clean = city_attraction.copy()
city_attraction_clean.set_index('label_id', inplace=True)
city_attraction_clean.drop(columns=['city', 'id'], inplace=True)
city_attraction_clean

In [ ]:

df = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_responses.csv')

In [ ]:
df.rename(columns = {'What country are you from? ': 'nationality', 'Age Range': 'age', 'Gender': 'gender',
                    '1. Choose your top favorite 3-5 cities you have traveled to that are on this list.  - Favorite City #1': 'favorite_city_one',
                    '2. Favorite city #2': 'favorite_city_two', '3. Favorite city #3': 'favorite_city_three',
                    '4. Favorite city #4': 'favorite_city_four', '5. Favorite city #5': 'favorite_city_five',
                    "6. If there's a city you have been and loved that is not on this list, add it below. ": 'extra_favorite',
                    "7. What cities on this list have you been to and not enjoyed?   - Least favorite city #1": 'least_favorite_one',
                    '8. Least favorite city #2': 'least_favorite_two', 
                    "9. If there's a city you have been to and haven't liked that is not on this list, add it below": 'extra_least_favorite',
                    "What price range of restaurant do you eat at when you travel? [Price level: 1 (Fast/Cheap Eats)]": 'food_one',
                    "What price range of restaurant do you eat at when you travel? [Price level: 2 (Casual Dining)]": 'food_two',
                    "What price range of restaurant do you eat at when you travel? [Price level: 3 (Upscale Dining)]": 'food_three',
                    "What price range of restaurant do you eat at when you travel? [Price level: 4 (Fine Dining/High End)]": 'food_four',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Art Gallery]": 'art_gallery',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Library]": 'library',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Museum ]": 'museum',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]": 'aquarium',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]": 'amusement_park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]": 'zoo',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]": 'movie_theater',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]": 'store',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]": 'park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]": 'natural_feature',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]": 'place_of_worship'}, inplace=True )

In [ ]:
df

In [ ]:
user_attraction = df[['nationality', 'age', 'gender', 'amusement_park', 'museum', 'park', 'art_gallery', 'aquarium', 'zoo', 
                     'library', 'movie_theater', 'natural_feature', 'place_of_worship', 'store']].copy()
user_attraction

In [ ]:
user_attraction.drop(columns=['nationality', 'age', 'gender'], inplace=True)

In [ ]:
replace_map = {'1 ( Would NOT go)': 1, '2': 2, '3': 3, '4 (Definitely would go)': 4}

user_attraction = user_attraction.replace(replace_map)


In [ ]:
user_attraction

In [ ]:
city_attraction_clean

In [ ]:
normalized_user_a = pd.DataFrame(pp.normalize(user_attraction))
normalized_user_a

In [ ]:
normalized_city_a = pd.DataFrame(pp.normalize(city_attraction_clean))
normalized_city_a

In [ ]:
sim_city_a = pd.DataFrame(cosine_similarity(normalized_city_a))

In [ ]:
cosine_sim_a = pd.DataFrame(cosine_similarity(normalized_user_a, normalized_city_a))
cosine_sim_a

In [ ]:
top_city_a = find_similar_n(cosine_sim_a,10)
top_city_a

### New dataframe build
Every cityxuser in row, 
Food sim, attraction sim, rank columns

In [ ]:
cities

In [ ]:
cities[cities['country'] == 'Germany']

In [ ]:
#Adding user_id to cityname 

lst = []
for i in range(153):
    for c in cities.city:

        a = c + "_" + str(i)
        lst.append(a)

In [ ]:
new = pd.DataFrame(lst)
new

In [ ]:
split = new[0].str.split("_", n = 1, expand = True)
split

In [ ]:
split.rename(columns={0:'city', 1:'user'}, inplace=True)
split

In [ ]:
sim_score_rank = pd.merge(left=split, right=food[['label_id', 'city']], on='city')
sim_score_rank.rename(columns={'label_id': 'city_id'}, inplace=True)

In [ ]:
food

In [ ]:
sim_score_rank

In [ ]:
sim_score_rank[sim_score_rank['city'] == 'Cancun']

In [ ]:
city_attraction_clean

In [ ]:
sim_score_rank['user'] = pd.to_numeric(sim_score_rank['user'])


In [ ]:
sim_score_rank.sort_values(['user', 'city_id'], inplace=True)

In [ ]:
lst = []
lst2 = []
lst3 = []
for user, city_id in zip(sim_score_rank.user, sim_score_rank.city_id):
    lst.append(cosine_sim.iloc[user, city_id])
    #lst2.append(user)
    #lst3.append(city_id)

In [ ]:
sim_score_rank['food_sim'] = lst

In [ ]:
sim_score_rank


In [ ]:
cosine_sim

In [ ]:
#Adding attraction sim scores for userxcity
lst_a = []
for user, city_id in zip(sim_score_rank.user, sim_score_rank.city_id):
        lst_a.append(cosine_sim_a.iloc[user, city_id])


In [ ]:
cosine_sim_a

In [ ]:
sim_score_rank['attraction_sim'] = lst_a

In [ ]:
sim_score_rank

## Here in staging

In [ ]:
top_city= df[['favorite_city_one', 'favorite_city_two', 'favorite_city_three', 'favorite_city_four', 'favorite_city_five']].copy()
top_city

In [ ]:
top_city.reset_index(inplace=True)
top_city.rename(columns={'index': 'user'}, inplace=True)

In [ ]:
top_city

In [ ]:
top_city_melt = top_city.melt(id_vars=['user'])

In [ ]:
top_city_melt

In [ ]:
def rank_from_col(x):
    if x.variable=='favorite_city_one':
       return 5
    elif x.variable=='favorite_city_two':
       return 4
    elif x.variable=='favorite_city_three':
       return 3
    elif x.variable=='favorite_city_four':
       return 2
    elif x.variable=='favorite_city_five':
       return 1
    elif x.value is None:
        return 0 
    


In [ ]:
top_city_melt['rank'] = top_city_melt.apply(rank_from_col,axis=1)


In [ ]:
top_city_melt.sort_values('user')

In [ ]:
top_city_no_na = top_city_melt.dropna().copy()

In [ ]:
top_city_no_na.sort_values('user')

In [ ]:
top_city_no_na.rename(columns={'value':'city'}, inplace=True)

In [ ]:
sim_score_rank.set_index(['user', 'city'], inplace=True)
top_city_no_na.set_index(['user', 'city'], inplace=True)

In [ ]:
top_city_no_na[top_city_no_na['rank'] == 5]

In [ ]:
top_city_no_na.loc[[4]]

In [ ]:
top_city_no_na

In [ ]:
sim_score_rank

In [ ]:
sim_score_rank = pd.merge(left=sim_score_rank, right=top_city_no_na[['rank']], left_index=True, right_index=True, how='left')


In [ ]:
sim_score_rank.fillna(0, inplace=True)

In [ ]:
sim_score_rank

In [ ]:
sim_score_rank[sim_score_rank['city_id'] == 23]

In [ ]:
sim_score_rank['sum'] = sim_score_rank['food_sim'] + sim_score_rank['attraction_sim']

In [ ]:
sim_score_rank

In [ ]:
top_10 = sim_score_rank.groupby('user').apply(lambda x: x.nlargest(10, 'sum')).droplevel(level=0)

In [ ]:
top_10[top_10['rank'] == 1]

In [ ]:
# Sort values by sum for each user group
ordered_sum = sim_score_rank.sort_values('sum', ascending=False).sort_index(level='user', sort_remaining=False)
ordered_sum

In [ ]:
ordered_sum['ranking_weight'] = ordered_sum['sum'] + ordered_sum['var']

In [ ]:
#Sort by ranking_weight for each user
ordered_sum = ordered_sum.sort_values('ranking_weight', ascending=False).sort_index(level='user', sort_remaining=False)

In [ ]:
len(ordered_sum.index.unique(level='user'))


In [ ]:
ordered_sum

In [ ]:
def find_top(n):
    top = ordered_sum.groupby('user').apply(lambda x: x.nlargest(n, 'sum')).droplevel(level=0)
    top_10_rank = top.loc[top['rank'].isin([1.0, 3.0, 2.0, 4.0, 5.0])]
    print( "number of users with a top city", len(top_10_rank.index.unique(level='user')))
    
    top['rank'].hist()
    print("number of ranked cities", np.count_nonzero(top['rank']))
    
    

In [ ]:
find_top(3)

In [ ]:
np.count_nonzero(ordered_sum['rank'])

In [ ]:
top_10 = ordered_sum.groupby('user').apply(lambda x: x.nlargest(10, 'sum')).droplevel(level=0)

#df.groupby('Brand').apply(lambda x: x.nlargest(2, 'Rank')).reset_index(drop=True)  


In [ ]:
top_10.info()

In [ ]:
np.count_nonzero(top_10['rank'])

In [ ]:
ordered_sum['ranking_weight'].hist()

### City to city sim scores

In [ ]:
together = sim_city_a + sim_city_food

In [ ]:
together

In [ ]:
together.index.name = 'foo'

In [ ]:
together.reset_index(inplace=True)

In [ ]:
together

In [ ]:
city_to_city_sim = together.melt('foo', var_name='city', value_name='sim_sum')

In [ ]:
city_to_city_sim.rename(columns={'foo': 'label_id'}, inplace=True)

In [ ]:
city_to_city_sim

In [ ]:
split[split['city'] == "Frankfurt am main"]

In [ ]:
food[food['city'] == 'Frankfurt']

In [ ]:
city_num = pd.merge(left=split, right=food[['label_id', 'city']], on='city')

In [ ]:
city_num.drop(columns=['user'], inplace=True)

In [ ]:
city_name_sim = city_to_city_sim.merge(city_num.drop_duplicates(), on=['label_id'], 
                   how='left')

In [ ]:
city_name_sim

In [ ]:
city_name_sim.rename(columns={'label_id': 'id_a', 'city_x': 'id_b', 'city_y': "city_a"}, inplace=True)

In [ ]:
city_num.rename(columns={'label_id':'id_b'}, inplace=True)

In [ ]:
city_name_sim = city_name_sim.merge(city_num.drop_duplicates(), on=['id_b'], 
                   how='left',)

In [ ]:
city_name_sim

In [ ]:
city_name_sim.rename(columns={'city': 'city_b'}, inplace=True)

In [ ]:
city_name_sim.reset_index(inplace=True, drop=True)

In [ ]:
city_name_sim

In [ ]:
city_name_sim.set_index(['id_b', 'id_a'], inplace=True)

In [ ]:
city_name_sim

In [ ]:
city_sim_final = city_name_sim.groupby('id_b').apply(pd.DataFrame.sort_values, 'sim_sum', ascending=False)

In [ ]:
city_sim_final

In [ ]:
city_sim_final.reset_index(level=0, drop=True, inplace=True)

In [ ]:
city_sim_final.xs(27, level='id_b', drop_level=False)

In [ ]:
city_sim_final.xs(11, level='id_b', drop_level=False)

In [ ]:
top_city_score = city_sim_final.groupby('id_b').apply(lambda x: x.nlargest(6, 'sim_sum')).droplevel(level=0)



In [ ]:
two = top_city_score[top_city_score['sim_sum'] == 2.000000]


In [ ]:
two

In [ ]:
list_int = [1,2,3,4,5,6]
top_city_score['list_int'] = np.tile(list_int, len(top_city_score)//len(list_int))

In [ ]:
top_city_score.head(19)

In [ ]:
top_city_score.pivot(values='city_a', index='city_b', columns='list_int')

In [ ]:
top_city_score.pivot(values='sim_sum', index='city_b', columns='list_int')

In [ ]:
top = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/top_city_num.csv", index_col=0)

In [ ]:
top

In [ ]:
cities_top = cities.copy()

In [ ]:
cities_top

In [ ]:
diff = set(cities.city).difference(set(food.city))
diff = list(diff)
print(len(diff))

In [ ]:
diff

In [ ]:

len(cities.city)

In [ ]:
food


In [ ]:
cities[cities['city'] == 'Guilin']